In [ ]:
# Code used for plotting data
# Author Samwel Portelli <samwel.portelli.18@um.edu.mt>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from deel.puncc.metrics import regression_ace
from deel.puncc.metrics import regression_mean_coverage
from deel.puncc.metrics import regression_sharpness
from deel.puncc.plotting import plot_prediction_intervals
import os
import csv

In [ ]:
models = ['arima', 'arima_garch', 'transformer_mcdropout', 'transformer_garch', 'transformer_enbpi']
alphas = [0.15, 0.1, 0.05, 0.01]

data_folders = ['\model_data_1','\model_data_2','\model_data_3','\model_data_5','\model_data_6','\model_data_OT']

In [ ]:
def load_data(model, alpha, s=None, data_folder='\model_data_OT'):
    
    folder_path = r'C:\Users\porte\Downloads' + data_folder
    
    if model=='transformer_enbpi':
        
        file_path = os.path.join(folder_path, f'transformer_confidence_run_data_s_{s}', f'ALL_PI_data_s_{s}.csv')
        file_path_true_pred = os.path.join(folder_path, f'transformer_confidence_run_data_s_{s}', f'y_true_and_pred_s_{s}.csv')
        og_data = r'C:\Users\porte\Downloads\dataset\exchange_rate\exchange_rate.csv'
        val_file_path = os.path.join(folder_path, f'transformer_confidence_run_data_s_{s}_val', f'ALL_PI_data_s_{s}.csv')
        val_file_path_true_pred = os.path.join(folder_path, f'transformer_confidence_run_data_s_{s}_val', f'y_true_and_pred_s_{s}.csv')
    
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, delimiter=',')

        # Iterate through each column and convert it to an array
        confidence_data = {}
        for column in df.columns:
            confidence_data[column] = df[column].to_numpy()

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path_true_pred, delimiter=',')

        # Iterate through each column and convert it to an array
        true_pred = {}
        for column in df.columns:
            true_pred[column] = df[column].to_numpy()

        # VALIDATION DATA

        # Read the CSV file into a DataFrame
        df = pd.read_csv(val_file_path, delimiter=',')

        # Iterate through each column and convert it to an array
        val_confidence_data = {}
        for column in df.columns:
            val_confidence_data[column] = df[column].to_numpy()

        # Read the CSV file into a DataFrame
        df = pd.read_csv(val_file_path_true_pred, delimiter=',')

        # Iterate through each column and convert it to an array
        val_true_pred = {}
        for column in df.columns:
            val_true_pred[column] = df[column].to_numpy()

        # ORIGINAL DATA

        # Read the CSV file into a DataFrame
        df = pd.read_csv(og_data, delimiter=',')

        # Iterate through each column and convert it to an array
        og_data = {}
        for column in df.columns:
            og_data[column] = df[column].to_numpy()

        #Test Set
        y_true=true_pred['y_true']
        y_pred=true_pred['y_pred_from_PUNCC']
        y_pred_lower=confidence_data['low_'+str(alpha)]
        y_pred_upper=confidence_data['high_'+str(alpha)]

        # Validation Set
        val_y_true=val_true_pred['y_true']
        val_y_pred=val_true_pred['y_pred_from_PUNCC']
        val_y_pred_lower=val_confidence_data['low_'+str(alpha)]
        val_y_pred_upper=val_confidence_data['high_'+str(alpha)]

        og_data_true=og_data[data_folder.split('_')[-1]]
        og_data_date=og_data['date']
    
    elif model=='transformer_garch':
        conf = int(100-alpha*100)
        
        file_path = os.path.join(folder_path, f'garch_transformer_confidence_run_data_alpha_{alpha}', f'garch_transforemer_y_true_and_pred_alp_{conf}_{data_folder.split("_")[-1]}.csv')
        #file_path = r'C:\Users\porte\Downloads'+str(data_folder)+'\\garch_transformer_confidence_run_data_alpha_'+str(alpha)+'\garch_transforemer_y_true_and_pred_alp_'+str(conf)+'.csv'

        df = pd.read_csv(file_path)

        y_true = df['y_true'].values
        y_pred = df['y_pred_from_PUNCC'].values
        y_pred_lower = df['y_lower'].values
        y_pred_upper = df['y_upper'].values

        file_path = os.path.join(folder_path, f'val_garch_transformer_confidence_run_data_alpha_{alpha}', f'val_garch_transforemer_y_true_and_pred_alp_{conf}_{data_folder.split("_")[-1]}.csv')
        #file_path = r'C:\Users\porte\Downloads'+str(data_folder)+'\\val_garch_transformer_confidence_run_data_alpha_'+str(alpha)+'\garch_transforemer_y_true_and_pred_alp_'+str(conf)+'.csv'

        df = pd.read_csv(file_path)

        val_y_true = df['y_true'].values
        val_y_pred = df['y_pred_from_PUNCC'].values
        val_y_pred_lower = df['y_lower'].values
        val_y_pred_upper = df['y_upper'].values

        og_data = r'C:\Users\porte\Downloads\dataset\exchange_rate\exchange_rate.csv'

        # Read the CSV file into a DataFrame
        df = pd.read_csv(og_data, delimiter=',')

        # Iterate through each column and convert it to an array
        og_data = {}
        for column in df.columns:
            og_data[column] = df[column].to_numpy()

        og_data_true=og_data[data_folder.split('_')[-1]]
        og_data_date=og_data['date'] 
        
    elif model=='transformer_mcdropout':
        conf = int(100-alpha*100)

        file_path = os.path.join(folder_path, 'output_mc_1_MCDROPOUT_PREDICTION', 'output_mc_1',f'min_max_mean_alpha_{alpha}.csv')
        
        df = pd.read_csv(file_path)

        file_path = os.path.join(folder_path, 'output_mc_1_MCDROPOUT_PREDICTION', 'output_mc_1','true_0.npy')
        
        y_true = np.load(file_path).flatten()
        y_pred = df['mean'].values
        y_pred_lower = df['lower'].values
        y_pred_upper = df['upper'].values


        file_path = os.path.join(folder_path, 'output_mc_1_MCDROPOUT_VAL', 'output_mc_1',f'min_max_mean_alpha_{alpha}.csv')
        
        df = pd.read_csv(file_path)

        file_path = os.path.join(folder_path, 'output_mc_1_MCDROPOUT_VAL', 'output_mc_1','true_0.npy')
        
        val_y_true = np.load(file_path).flatten()
        
        val_y_pred = df['mean'].values
        val_y_pred_lower = df['lower'].values
        val_y_pred_upper = df['upper'].values

        og_data = r'C:\Users\porte\Downloads\dataset\exchange_rate\exchange_rate.csv'

        # Read the CSV file into a DataFrame
        df = pd.read_csv(og_data, delimiter=',')

        # Iterate through each column and convert it to an array
        og_data = {}
        for column in df.columns:
            og_data[column] = df[column].to_numpy()

        og_data_true=og_data[data_folder.split('_')[-1]]
        og_data_date=og_data['date']        
    elif model=='arima_garch':
        conf = int(100-alpha*100)

        data_file_path = r'C:\Users\porte\Downloads'+str(data_folder)+'\\arimagarch_y_true_and_pred_alp_'+str(conf)+'.csv'

        # Read data CSV file
        df_data = pd.read_csv(data_file_path, delimiter=',')

        # Store columns in arrays
        y_true = df_data['y_true'].values
        y_pred = df_data['y_pred_from_arimagarch'].values 
        y_pred_lower = df_data['y_lower'].values
        y_pred_upper = df_data['y_upper'].values

        data_file_path = r'C:\Users\porte\Downloads'+str(data_folder)+'\\val_arimagarch_y_true_and_pred_alp_'+str(conf)+'.csv'

        # Read data CSV file
        df_data = pd.read_csv(data_file_path, delimiter=',')

        # Store columns in arrays
        val_y_true = df_data['y_true'].values
        val_y_pred = df_data['y_pred_from_arimagarch'].values 
        val_y_pred_lower = df_data['y_lower'].values
        val_y_pred_upper = df_data['y_upper'].values

        og_data = r'C:\Users\porte\Downloads\dataset\exchange_rate\exchange_rate.csv'

        # Read the CSV file into a DataFrame
        df = pd.read_csv(og_data, delimiter=',')

        # Iterate through each column and convert it to an array
        og_data = {}
        for column in df.columns:
            og_data[column] = df[column].to_numpy()

        og_data_true=og_data[data_folder.split('_')[-1]]
        og_data_date=og_data['date']        
    elif model=='arima':
        conf = int(100-alpha*100)

        data_file_path = r'C:\Users\porte\Downloads'+str(data_folder)+'\\arima_y_true_and_pred_alp_'+str(conf)+'.csv'

        # Read data CSV file
        df_data = pd.read_csv(data_file_path, delimiter=',')

        # Store columns in arrays
        y_true = df_data['y_true'].values
        y_pred = df_data['y_pred_from_arima'].values 
        y_pred_lower = df_data['y_lower'].values
        y_pred_upper = df_data['y_upper'].values

        data_file_path = r'C:\Users\porte\Downloads'+str(data_folder)+'\\val_arima_y_true_and_pred_alp_'+str(conf)+'.csv'

        # Read data CSV file
        df_data = pd.read_csv(data_file_path, delimiter=',')

        # Store columns in arrays
        val_y_true = df_data['y_true'].values
        val_y_pred = df_data['y_pred_from_arima'].values 
        val_y_pred_lower = df_data['y_lower'].values
        val_y_pred_upper = df_data['y_upper'].values

        og_data = r'C:\Users\porte\Downloads\dataset\exchange_rate\exchange_rate.csv'

        # Read the CSV file into a DataFrame
        df = pd.read_csv(og_data, delimiter=',')

        # Iterate through each column and convert it to an array
        og_data = {}
        for column in df.columns:
            og_data[column] = df[column].to_numpy()

        og_data_true=og_data[data_folder.split('_')[-1]]
        og_data_date=og_data['date']        
    else:
        print('Model does not exist. Choose a suitable model!')
        
    return y_true, y_pred, y_pred_lower, y_pred_upper, val_y_true, val_y_pred, val_y_pred_lower, val_y_pred_upper, og_data_true, og_data_date


In [ ]:
for alpha in alphas:
    
    per_signal_ace_list, per_signal_ac_list, per_signal_pi_width_list, model_names_list_list = [], [], [], []
    
    for data_folder in data_folders:
    
        ace_list, ac_list, pi_width_list, model_names_list = [], [], [], []
        
        for model in models:

            if model == 'transformer_enbpi':
                ss = [5, 10, 20, 25, 50, 100]
            else:
                ss = [None]

            for s in ss:

                print(f"Model: {model}, s: {s}, Alpha: {alpha}, Data Folder: {data_folder}")

                # Loading data
                y_true, y_pred, y_pred_lower, y_pred_upper, val_y_true, val_y_pred, val_y_pred_lower, val_y_pred_upper, og_data_true, og_data_date = load_data(model=model, alpha=alpha, s=s, data_folder=data_folder)

                ace = regression_ace(y_true, y_pred_lower, y_pred_upper, alpha)
                ac = regression_mean_coverage(y_true, y_pred_lower, y_pred_upper)
                pi_width = regression_sharpness(y_pred_lower, y_pred_upper)
                
                ace_list.append(ace)
                ac_list.append(ac)
                pi_width_list.append(pi_width)
                model_names_list.append(model + '_' + str(s))
                
                
                
        per_signal_ace_list.append(np.concatenate((np.array([data_folder.split('_')[-1]]), ace_list)))
        per_signal_ac_list.append(np.concatenate((np.array([data_folder.split('_')[-1]]), ac_list)))
        per_signal_pi_width_list.append(np.concatenate((np.array([data_folder.split('_')[-1]]), pi_width_list)))
        model_names_list_list.append(np.concatenate((np.array(['Metric']), model_names_list)))
    
    def save_lists_to_csv(list1, list_of_lists, filename):
        with open(filename, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)

            # Write data
            for i, (item1, sublist) in enumerate(zip(list1, zip(*list_of_lists))):
                row = [item1] + list(sublist)
                csv_writer.writerow(row)

    ace_filename = 'ace_' + str(alpha) + '.csv' 
    ac_filename = 'ac_' + str(alpha) + '.csv' 
    pi_width_filename = 'pi_width_' + str(alpha) + '.csv' 
    
    save_lists_to_csv(model_names_list_list[-1], per_signal_ac_list, ac_filename)
    save_lists_to_csv(model_names_list_list[-1], per_signal_ace_list, ace_filename)
    save_lists_to_csv(model_names_list_list[-1], per_signal_pi_width_list, pi_width_filename)


In [ ]:
for data_folder in data_folders:

    for alpha in alphas:
        
        ace_list, ac_list, pi_width_list, model_names_list = [], [], [], []
        
        for model in models:

            if model == 'transformer_enbpi':
                ss = [5, 10, 20, 25, 50, 100]
            else:
                ss = [None]

            for s in ss:

                print(f"Model: {model}, s: {s}, Alpha: {alpha}, Data Folder: {data_folder}")

                # Loading data
                y_true, y_pred, y_pred_lower, y_pred_upper, val_y_true, val_y_pred, val_y_pred_lower, val_y_pred_upper, og_data_true, og_data_date = load_data(model=model, alpha=alpha, s=s, data_folder=data_folder)

                ace = regression_ace(y_true, y_pred_lower, y_pred_upper, alpha)
                ac = regression_mean_coverage(y_true, y_pred_lower, y_pred_upper)
                pi_width = regression_sharpness(y_pred_lower, y_pred_upper)
                
                ace_list.append(ace)
                ac_list.append(ac)
                pi_width_list.append(pi_width)
                model_names_list.append(model + '_' + str(s))
                
                
                
        
        # Combine the lists
        data = zip(model_names_list, ac_list, ace_list, pi_width_list)

        # Specify the file name
        filename = ".\\UpdatedPlots\\v1_exp1_metrics_" +str(int((1-alpha)*100))+ ".csv"

        # Write the data to a CSV file
        with open(filename, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Model + UQ', 'AC', 'ACE', 'PI-Width'])  # Writing header
            csvwriter.writerows(data)


**Metrics for the each UQ**

- AC
- ACE
- PI-Width

*Plotting the metrics*

In [ ]:
def returnCurrency(data_folder):
    signal = data_folder.split('_')[-1]
    
    if signal == 'OT':
        return 'GBP/USD'
    elif signal == '1':
        return 'SGD/USD'
    elif signal == '2':
        return 'JPY/USD'
    elif signal == '3':
        return 'CNY/USD'
    elif signal == '5':
        return 'CAD/USD'
    elif signal == '6':
        return 'AUD/USD'

In [ ]:
data_folders = ['\model_data_1','\model_data_2','\model_data_3','\model_data_5','\model_data_6','\model_data_OT']
model_names = ['ARIMA', 'ARIMA+GARCH', 'Transformer+MC-Dropout', 'Transformer+GARCH', 'Transformer+EnbPI (s=5)', 'Transformer+EnbPI (s=10)', 'Transformer+EnbPI (s=20)', 'Transformer+EnbPI (s=25)', 'Transformer+EnbPI (s=50)', 'Transformer+EnbPI (s=100)']

for data_folder in data_folders:
    
    ace_list_list, ac_list_list, pi_width_list_list = [], [], []
    
    for model in models:
        
        

        if model == 'transformer_enbpi':
            ss = [5, 10, 20, 25, 50, 100]
        else:
            ss = [None]

        for s in ss:
            
            ace_list, ac_list, pi_width_list = [], [], []

            for alpha in alphas:
                
                print(f"Model: {model}, s: {s}, Alpha: {alpha}, Data Folder: {data_folder}")

                # Loading data
                y_true, y_pred, y_pred_lower, y_pred_upper, val_y_true, val_y_pred, val_y_pred_lower, val_y_pred_upper, og_data_true, og_data_date = load_data(model=model, alpha=alpha, s=s, data_folder=data_folder)

                ace = regression_ace(y_true, y_pred_lower, y_pred_upper, alpha)
                ac = regression_mean_coverage(y_true, y_pred_lower, y_pred_upper)
                pi_width = regression_sharpness(y_pred_lower, y_pred_upper)
                
                ace_list.append(ace)
                ac_list.append(ac)
                pi_width_list.append(pi_width)
                

                
            ace_list_list.append(np.array(ace_list))
            ac_list_list.append(np.array(ac_list))
            pi_width_list_list.append(np.array(pi_width_list))

    plt.title('Average Coverage vs. Alpha When Forecasting ' +returnCurrency(data_folder))
    plt.xlabel(r'$\alpha$')
    plt.ylabel('Coverage')
    plt.grid()

    for ind, ac in enumerate(ac_list_list):

        conf = np.subtract(np.ones(4), alphas)
        plt.plot(alphas, ac, label=model_names[ind], marker='o')

    plt.plot(alphas, conf, label='Actual Confidence', linestyle='--', color='black', marker='^', alpha=0.3)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13), fancybox=True, shadow=False, ncol=3)
    plt.xticks(alphas)
    plt.savefig('.\\UpdatedPlots\\ACvsAlpha_'+data_folder.split('_')[-1]+'.png', bbox_inches='tight')
    
    plt.show()

    plt.title('PI-Width vs. Alpha When Forecasting ' +returnCurrency(data_folder))
    plt.xlabel(r'$\alpha$')
    plt.ylabel('PI-Width')
    plt.grid()
    plt.xticks(alphas)

    for ind, pi_width in enumerate(pi_width_list_list):

        conf = np.subtract(np.ones(4), alphas)
        plt.plot(alphas, pi_width, label=model_names[ind], marker='o')

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.13), fancybox=True, shadow=False, ncol=3)
    plt.savefig('.\\UpdatedPlots\\PIvsAlpha_'+data_folder.split('_')[-1]+'.png', bbox_inches='tight')
    plt.show()
    

*Plotting the volatility against the uncertatinty*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Generate sample time-series data
np.random.seed(0)
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))

# Define the window size for rolling volatility calculation
window = 5

# Calculate rolling volatility using the standard deviation
rolling_volatility_std = ts.rolling(window).std()

# Plotting
plt.figure(figsize=(10, 6))

ts.plot(color='gray', label='Original Time-Series')
rolling_volatility_std.plot(color='blue', label='Rolling Volatility (Standard Deviation)')

plt.title('Rolling Volatility of a Time-Series')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def save_arrays_to_csv(model_name, s, alpha, uncertainty, std_volatility):
    # Create the file name based on the model, s, and alpha
    file_name = f".\\UpdatedPlots\\uncertainty_vs_volatility\\uncertainty_vs_volatility_{model_name}_s{s}_alpha{alpha}.csv"
    
    df = pd.DataFrame({"uncertainty" : uncertainty, "std_volatility" : std_volatility})
    df.to_csv(file_name, index=False)

In [ ]:
import pandas as pd
vol_window = 5
model_names = ['ARIMA', 'ARIMA+GARCH', 'Transformer+MC-Dropout', 'Transformer+GARCH', 'Transformer+EnbPI (s=5)', 'Transformer+EnbPI (s=10)', 'Transformer+EnbPI (s=20)', 'Transformer+EnbPI (s=25)', 'Transformer+EnbPI (s=50)', 'Transformer+EnbPI (s=100)']
ace_list_list, ac_list_list, pi_width_list_list = [], [], []



for data_folder in data_folders:
    i=0
    
    for ind, model in enumerate(models):
        
        

        if model == 'transformer_enbpi':
            ss = [5, 10, 20, 25, 50, 100]
        else:
            ss = [None]

        for s in ss:
            
            ace_list, ac_list, pi_width_list = [], [], []

           
            
            for alpha in alphas:
                
                print(f"Model: {model}, s: {s}, Alpha: {alpha}, Data Folder: {data_folder}")

                # Loading data
                y_true, y_pred, y_pred_lower, y_pred_upper, val_y_true, val_y_pred, val_y_pred_lower, val_y_pred_upper, og_data_true, og_data_date = load_data(model=model, alpha=alpha, s=s, data_folder=data_folder)

                uncertainty = np.subtract(y_pred_upper, y_pred_lower)
                
                y_true_df = pd.DataFrame(y_true)
                
                std_volatility = y_true_df.rolling(vol_window).std()
                
                fig, ax1 = plt.subplots(figsize=(10,3))

                ax1.plot(uncertainty, label='Model Uncertainty', color='blue',alpha=0.5, linewidth=0.8)
                ax1.set_ylabel('Model Uncertainty')
                ax1.set_xlim(0, len(uncertainty))
                ax1.set_xlabel('Index')

                ax2 = ax1.twinx()
                ax2.plot(std_volatility, label='Volatility', color='red',alpha=0.5, linewidth=0.8)
                ax2.set_ylabel('Volatility')
                plt.xlabel('Index')
                plt.grid(True)

                fig.legend(loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.17))
                #plt.title(f"Model: {model}, s: {s}, Alpha: {alpha}")
                plt.title(f"Model Uncertainty ({model_names[i]},α={alpha}) and Volatility for GBP/USD")
                filename = '.\\UpdatedPlots\\uncert_and_vol_' + model + '_s' +str(s)+ '_alpha_' + str(alpha) + '.png'  
                plt.savefig(filename, bbox_inches='tight')                
                plt.show()
                
                save_arrays_to_csv(model, s, alpha, uncertainty, std_volatility.values.flatten())
            
            i += 1
    
                

*Table of portfolio matrix*

In [ ]:
signals = ['1', '2', '3', '5', '6', 'OT']
model = 'arima_garch'
s = None


for signal in signals:

    initial = True
    table = []
    
    for strategy in ['1', '3']:

        for alpha in alphas:
            
            # File paths
            metrics_path = f"C:\\Users\\porte\\Downloads\\Trading data with updated garch and commission\\trading_model_data_{signal}\\{model}_alpha{str(alpha)}_s{s}_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

            # Read CSV files as DataFrames
            metrics_df = pd.read_csv(metrics_path)
            
            if initial:
                table.append(np.concatenate((np.array(['Metrics']), np.array(metrics_df['Metric']))))
                initial = False
           
            table.append(np.concatenate((np.array([str(alpha)]), np.array(metrics_df['Mean']))))

    # Transposing the array
    data_transposed = np.array(table).T.tolist()

    # Writing to CSV
    filename = model + '_portfoliometrics_' + signal +'_v2.csv'
    
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data_transposed)
                

plot of sharpe ratio against alpha for all the signals

In [ ]:
# SHORT ONLY
signals = ['1', '2', '3', '5', '6', 'OT']

metrics = ['Cumulative Returns', 'Annualized Volatility', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown']

model = 'arima_garch'
s = None

model = 'transformer_enbpi'
s = 20

strategy_name = ['Long', 'Short']
signal_name = ['SGD/USD', 'JPY/USD', 'CNY/USD', 'CAD/USD', 'AUD/USD', 'GBP/USD', 'Average Line']



for indx, metric in enumerate(metrics):
    
    fig, ax1 = plt.subplots(1, 1, figsize=(5, 4)) 
    
    for ind, strategy in enumerate(['1', '3']):

        i=0
        data = []

        for signal in signals:

            table = []

            for alpha in alphas:

                # File paths
                metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{str(alpha)}_s{s}_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

                # Read CSV files as DataFrames
                metrics_df = pd.read_csv(metrics_path)
                
                table.append(metrics_df['Mean'][indx])

            data.append(np.array(table))

            #if strategy == '1':
            #    ax1.plot(alphas, table, label=signal_name[i], marker='o')
            #    ax1.grid(True)
            #    ax1.set_title('Sharpe Ratio Across α \n for Long Strategy Using Model Certainty')
            #    ax1.set_xlabel(r'$\alpha$')
            #    ax1.set_ylabel('Sharpe Ratio')
            if strategy == '3':
                ax1.plot(alphas, table, label=signal_name[i], marker='o')
                ax1.grid(True)
                ax1.set_title(metric + ' Across α \n ARIMA + GARCH Model')
                ax1.set_xlabel(r'$\alpha$')
                ax1.set_ylabel(metric)

            i += 1

        #if strategy == '1':
        #    ax1.plot(alphas, np.mean(data, axis=0), label='Mean', marker='^', linestyle='--', color='black')
        #    ax1.set_xticks(alphas)
        if strategy == '3':
            ax1.plot(alphas, np.mean(data, axis=0), label='Mean', marker='^', linestyle='--', color='black')
            ax1.set_xticks(alphas)
        
    fig.legend(signal_name, loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.23))
    plt.savefig('.\\UpdatedPlots\\ARIMA'+metric.replace(" ", "")+'vsAlpha.png', bbox_inches='tight')
    plt.show()
                

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
alphas = [0.15, 0.1, 0.05, 0.01]

# SHORT ONLY
signals = ['1', '2', '3', '5', '6', 'OT']
metrics = ['Cumulative Returns', 'Annualized Volatility', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown']

model = 'arima_garch'
s = None

model = 'transformer_enbpi'
s = 20

strategy_name = ['Long', 'Short']
signal_name = ['SGD/USD', 'JPY/USD', 'CNY/USD', 'CAD/USD', 'AUD/USD', 'GBP/USD', 'Average Line']

# Create subplots: 3 rows, 2 columns
fig, axs = plt.subplots(3, 2, figsize=(10, 12))#, gridspec_kw={'height_ratios': [1, 1, 1]})

# Loop through metrics
for metric_ind, metric in enumerate(metrics):

    data = []

    for ind, strategy in enumerate(['1', '3']):
        i = 0
        for signal in signals:

            table = []

            for alpha in alphas:
                # File paths
                metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{str(alpha)}_s{s}_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

                # Read CSV files as DataFrames
                metrics_df = pd.read_csv(metrics_path)

                table.append(metrics_df['Mean'][metric_ind])

            data.append(np.array(table))

            row, col = divmod(metric_ind, 2)  # Determine subplot position

            if strategy == '3':
                axs[row, col].plot(alphas, table, label=signal_name[i], marker='o')
                axs[row, col].grid(True)
                axs[row, col].set_title(metric + ' Across α \n Transformer + EnbPI Model')
                axs[row, col].set_xlabel(r'$\alpha$')
                axs[row, col].set_ylabel(metric)
            
            i += 1

    # Plot the mean line for strategy '3'
    row, col = divmod(metric_ind, 2)
    axs[row, col].plot(alphas, np.mean(data, axis=0), label='Mean', marker='^', linestyle='--', color='black')
    axs[row, col].set_xticks(alphas)

# Remove the last empty subplot and center the last plot
fig.delaxes(axs[2, 1])

# Adjust the width of the third row plot to span both columns
axs[2, 0].set_position([0.2, 0, 0.5, 0.2])  # Manually adjust position: [left, bottom, width, height]

fig.subplots_adjust(hspace=0.5, wspace=0.3)

# Legend and layout
fig.legend(signal_name, loc='lower center', ncol=3)
plt.savefig('.\\UpdatedPlots\\TEmetricsvsAlpha.png', bbox_inches='tight')
plt.show()


buy and hold metrics table

In [ ]:
def calculate_portfolio_metrics(df, plot=True, havedate=True):
        
    if havedate:
        # Converting the date column to datetime
        df['Date'] = pd.to_datetime(df['Date'])
    
    # Calculating the daily returns
    df['daily_returns'] = df['portfolio_value'].pct_change()

    # Calculating the cumulative returns
    cumulative_returns = df['portfolio_value'].iloc[-1]/df['portfolio_value'].iloc[0] - 1
    
    annual_returns = (1+cumulative_returns)**(365/len(df['daily_returns'])) - 1 #https://www.investopedia.com/terms/a/annualized-total-return.asp

    # Calculating the annualized volatility
    annual_volatility = df['daily_returns'].std()*np.sqrt(len(df['daily_returns']))

    # Risk-free rate is assumed to be 1%
    risk_free_rate = 0.01/365
    
    sharpe_ratio = (annual_returns - risk_free_rate) / annual_volatility

    # Calculating the sortino ratio
    negative_returns = df[df['daily_returns']<0]['daily_returns']
    downside_volatility = negative_returns.std()*np.sqrt(len(df['daily_returns']))
    sortino_ratio = (annual_returns - risk_free_rate) / downside_volatility

    # Calculating the beta alpha using linear regression
    #benchmark_returns = SandPData.pct_change()[1:]
    #(beta, alpha) = stats.linregress(benchmark_returns, df['daily_returns'][1:])[0:2]
    #benchmark_returns = pd.Series(buy_and_hold(og_data['tradeactualPrice'])).pct_change()[1:]
    #(beta, alpha) = stats.linregress(benchmark_returns, df['daily_returns'][1:])[0:2]

    # Calculating the maximum drawdown
    df['cumulative_returns'] = df['portfolio_value']/df['portfolio_value'].iloc[0]
    df['peak'] = df['cumulative_returns'].cummax()
    df['drawdown'] = df['cumulative_returns'] - df['peak']
    max_drawdown = df['drawdown'].min()
    
    if plot:
        # Plotting the portfolio value
        sns.set(style='whitegrid')
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['portfolio_value'], linewidth=2)
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value')
        plt.title('Portfolio Growth')
        plt.xticks(rotation=45)
        plt.xlim(df['Date'].iloc[0], df['Date'].iloc[-1]) 

        
        y_min = df['portfolio_value'].min() * 0.95  
        y_max = df['portfolio_value'].max() * 1.05  
        plt.ylim(y_min, y_max)

        plt.tight_layout()
        sns.despine()
        plt.savefig("portfolio_growth.png")
        plt.show()

        # Plotting maximum drawdown
        plt.figure(figsize=(12, 6))
        plt.plot(df['Date'], df['drawdown'], color='red', linewidth=2)
        plt.fill_between(df['Date'], df['drawdown'], color='lightcoral')  
        plt.xlabel('Date')
        plt.ylabel('Drawdown')
        plt.title('Maximum Drawdown')
        plt.xticks(rotation=45)
        plt.xlim(df['Date'].iloc[0], df['Date'].iloc[-1])  

        plt.tight_layout()
        sns.despine()
        plt.savefig("maximum_drawdown.png")
        plt.show()

    # Removing temporary columns
    df.drop(['daily_returns','cumulative_returns', 'peak', 'drawdown'], axis=1, inplace=True)

    #return annual_returns, cumulative_returns, annual_volatility, sharpe_ratio, sortino_ratio, max_drawdown, beta, alpha
    return cumulative_returns, annual_volatility, sharpe_ratio, sortino_ratio, max_drawdown

In [ ]:
def buy_and_hold(priceData, initialCapital=1000, do_print=False, compute_metrics=False):

    returns = priceData.pct_change()

    returnsDf = pd.DataFrame(returns)
    returnsDf.columns = ['returns']
    
    returnsDf = returnsDf.reset_index(drop=True)

    fund_value = list()
    fund_value.append(initialCapital-initialCapital*0.0001*0.2)

    for i in range(len(priceData)):
        if i>0:
            if i == len(priceData)-1:
                fund_value.append(fund_value[i-1]+fund_value[i-1]*returnsDf['returns'][i] + (fund_value[i-1]+fund_value[i-1]*returnsDf['returns'][i])*0.0001*0.2)
            else:
                fund_value.append(fund_value[i-1]+fund_value[i-1]*returnsDf['returns'][i])
        
    
    if compute_metrics:
        cumulative_returns, annual_volatility, sharpe_ratio, sortino_ratio, max_drawdown = calculate_portfolio_metrics(pd.DataFrame(fund_value, columns=['portfolio_value']), plot=False, havedate=False)
    
    if do_print:
        print("Cumulative Returns:", cumulative_returns)
        print("Annualized Volatility:", annual_volatility)
        print("Sharpe Ratio:", sharpe_ratio)
        print("Sortino Ratio:", sortino_ratio)
        print("Maximum Drawdown:", max_drawdown)
        
        return cumulative_returns, annual_volatility, sharpe_ratio, sortino_ratio, max_drawdown
    else:   
        return fund_value

In [ ]:
data = pd.read_csv(r"C:\Users\porte\Downloads\dataset\exchange_rate\exchange_rate.csv")

In [ ]:
signals = ['1', '2', '3', '5', '6', 'OT']

table = []
table.append(['Metrics', 'Cumulative Returns', 'Annualized Volatility', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown'])

for ind, signal in enumerate(signals):
    
    cumulative_returns, annual_volatility, sharpe_ratio, sortino_ratio, max_drawdown = buy_and_hold(data[signal][-1518:], initialCapital=1000000, do_print=True, compute_metrics=True)
    
    table.append([signal_name[ind], cumulative_returns, annual_volatility, sharpe_ratio, sortino_ratio, max_drawdown])

# Transposing the array
data_transposed = np.array(table).T.tolist()

# Writing to CSV
filename = 'buyandhold.csv'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_transposed)

In [ ]:

signal = 'OT'
model = 'transformer_enbpi'
model_title = 'Transformer + EnbPI'
s = 20
alphas = [0.15, 0.1, 0.05, 0.01]

#model = 'arima_garch'
#model_title = 'ARIMA + GARCH'
#s = None

strategies = ['Long', 'Longusingthresholdedcertainty', 'Short', 'Shortusingthresholdedcertainty']
strategieslegend = ['Long', 'Long using certainty', 'Short', 'Short using certainty']

for alpha in alphas:
    
    #fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10, 10))
    #fig.subplots_adjust(wspace=0.3)  
    
    values = []
    drawdown = []
    
    alpha = str(alpha)
    
    for strategy in strategies:

        # File paths
        portfolio_value_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{alpha}_s{s}_optimisesharpe\\portfolio_valueDf_{strategy}.csv"

        # Read CSV files as DataFrames
        df = pd.read_csv(portfolio_value_path)
        
        # Calculating the maximum drawdown
        df['cumulative_returns'] = df['portfolio_value']/df['portfolio_value'].iloc[0]
        df['peak'] = df['cumulative_returns'].cummax()
        df['drawdown'] = df['cumulative_returns'] - df['peak']
        max_drawdown = df['drawdown'].min()
        
        values.append(np.array(df['portfolio_value']))
        drawdown.append(np.array(df['drawdown']))
        
    for i in range(len(strategieslegend)):
        
        if i == 0:
            
            fig, ax1 = plt.subplots(figsize=(5, 3))
            #ax1.plot(values[i], label=strategieslegend[i], color='blue', linewidth=0.9, linestyle='dotted')
            #ax1.plot(values[i+1], label=strategieslegend[i+1], color='blue', linewidth=0.7)
            ax1.plot(values[i+2], label=strategieslegend[i+2], color='black', linewidth=0.9, linestyle='dotted')
            ax1.plot(values[i+3], label=strategieslegend[i+3], color='black', linewidth=0.7)
            ax1.set_title('Equity Curve From ' + model_title + r' with $\alpha$=' + alpha)
            ax1.set_ylabel('Value ($)')
            ax1.set_xlabel('Index')
            ax1.set_xlim(0, len(values[i]))
            ax1.grid(True)
            fig.legend(['Without uncertainty', 'With certainty'], loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.23))
            filename = '.\\UpdatedPlots\\transformer_equity_' + alpha + '_' + model + '.png' 
            plt.savefig(filename, bbox_inches='tight')
            

            # Create a second y-axis sharing the same x-axis
            fig, ax3 = plt.subplots(figsize=(5, 3))
            ax3.set_title('Drawdown From ' + model_title + r' with $\alpha$=' + alpha)
            ax3.fill_between(np.arange(0, len(drawdown[i+2])), drawdown[i+2], label='Without uncertainty', color='grey', alpha=0.3, linestyle='--', linewidth=1.5)
            ax3.fill_between(np.arange(0, len(drawdown[i+3])), drawdown[i+3], label='With certainty', color='grey', alpha=0.6, linewidth=1.5)
            ax3.set_ylabel('Drawdown')
            ax3.set_xlabel('Index')
            ax3.set_xlim(0, len(drawdown[i]))
            ax3.set_ylim(min(drawdown[i+2])*1.1, 0)
            ax3.grid(True)
            fig.legend(['Without uncertainty', 'With certainty'], loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.23))
            filename = '.\\UpdatedPlots\\transformer_drawdown_' + alpha + '_' + model + '.png' 
            plt.savefig(filename, bbox_inches='tight')
            
    plt.show()
        

In [ ]:

signal = 'OT'
model = 'arima_garch'
model_title = 'ARIMA + GARCH'
s = None

#model = 'arima_garch'
#model_title = 'ARIMA + GARCH'
#s = None

strategies = ['Long', 'Longusingthresholdedcertainty', 'Short', 'Shortusingthresholdedcertainty']
strategieslegend = ['Long', 'Long using certainty', 'Short', 'Short using certainty']

for alpha in alphas:
    
    #fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10, 10))
    #fig.subplots_adjust(wspace=0.3)  
    
    values = []
    drawdown = []
    
    alpha = str(alpha)
    
    for strategy in strategies:

        # File paths
        portfolio_value_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{alpha}_s{s}_optimisesharpe\\portfolio_valueDf_{strategy}.csv"

        # Read CSV files as DataFrames
        df = pd.read_csv(portfolio_value_path)
        
        # Calculating the maximum drawdown
        df['cumulative_returns'] = df['portfolio_value']/df['portfolio_value'].iloc[0]
        df['peak'] = df['cumulative_returns'].cummax()
        df['drawdown'] = df['cumulative_returns'] - df['peak']
        max_drawdown = df['drawdown'].min()
        
        values.append(np.array(df['portfolio_value']))
        drawdown.append(np.array(df['drawdown']))
        
    for i in range(len(strategieslegend)):
        
        if i == 0:
            
            fig, ax1 = plt.subplots(figsize=(5, 3))
            #ax1.plot(values[i], label=strategieslegend[i], color='blue', linewidth=0.9, linestyle='dotted')
            #ax1.plot(values[i+1], label=strategieslegend[i+1], color='blue', linewidth=0.7)
            ax1.plot(values[i+2], label=strategieslegend[i+2], color='black', linewidth=0.9, linestyle='dotted')
            ax1.plot(values[i+3], label=strategieslegend[i+3], color='black', linewidth=0.7)
            ax1.set_title('Equity Curve From ' + model_title + r' with $\alpha$=' + alpha)
            ax1.set_ylabel('Value ($)')
            ax1.set_xlabel('Index')
            ax1.set_xlim(0, len(values[i]))
            ax1.grid(True)
            fig.legend(['Without uncertainty', 'With certainty'], loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.23))
            filename = '.\\UpdatedPlots\\equity_' + alpha + '_' + model + '.png' 
            plt.savefig(filename, bbox_inches='tight')
            
            #ax1.fill_between(np.arange(0, len(drawdown[i])), drawdown[i], label=strategieslegend[i], color='lightcoral')
            #ax3.set_title('Drawdown for ' + model_title + r' with $\alpha$=' + alpha)
            #ax3.legend()
            #ax3.set_ylabel('Drawdown')
            #ax3.set_xlabel('Index')
            #ax3.grid(True)
            
            # Create a second y-axis sharing the same x-axis
            fig, ax3 = plt.subplots(figsize=(5, 3))
            ax3.set_title('Drawdown From ' + model_title + r' with $\alpha$=' + alpha)
            #ax3.fill_between(np.arange(0, len(drawdown[i])), drawdown[i], label='Long', color='blue', alpha=0.3, linestyle='--', linewidth=1.5)
            #ax3.fill_between(np.arange(0, len(drawdown[i+1])), drawdown[i+1], label='Long with certainty', color='blue', alpha=0.6, linewidth=1.5)
            ax3.fill_between(np.arange(0, len(drawdown[i+2])), drawdown[i+2], label='Without uncertainty', color='grey', alpha=0.3, linestyle='--', linewidth=1.5)
            ax3.fill_between(np.arange(0, len(drawdown[i+3])), drawdown[i+3], label='With certainty', color='grey', alpha=0.6, linewidth=1.5)
            ax3.set_ylabel('Drawdown')
            ax3.set_xlabel('Index')
            ax3.set_xlim(0, len(drawdown[i]))
            ax3.set_ylim(min(drawdown[i+2])*1.1, 0)
            ax3.grid(True)
            fig.legend(['Without uncertainty', 'With certainty'], loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.23))
            filename = '.\\UpdatedPlots\\drawdown_' + alpha + '_' + model + '.png' 
            plt.savefig(filename, bbox_inches='tight')
            
    plt.show()
        

In [ ]:
signals = ['1', '2', '3', '5', '6', 'OT']
model = 'arima_garch'
s = None


for signal in signals:

    initial = True
    table = []
    
    for strategy in ['3']:

        for alpha in alphas:
            
            # File paths
            metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{str(alpha)}_s{s}_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

            # Read CSV files as DataFrames
            metrics_df = pd.read_csv(metrics_path)
            
            if initial:
                table.append(np.concatenate((np.array(['Metrics']), np.array(metrics_df['Metric']))))
                initial = False
           
            table.append(np.concatenate((np.array([str(alpha)]), np.array(metrics_df['Mean']))))

    # Transposing the array
    data_transposed = np.array(table).T.tolist()

    # Writing to CSV
    filename = '.\\UpdatedPlots\\' + model + '_portfoliometrics_' + signal +'_v2.csv'
    
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data_transposed)
                

In [ ]:
signals = ['1', '2', '3', '5', '6', 'OT']
model = 'transformer_enbpi'
alphas = [0.15, 0.1, 0.05, 0.01]
s = 20


for signal in signals:

    initial = True
    table = []
    
    for strategy in ['3']:

        for alpha in alphas:
            
            # File paths
            metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{str(alpha)}_s{s}_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

            # Read CSV files as DataFrames
            metrics_df = pd.read_csv(metrics_path)
            
            if initial:
                table.append(np.concatenate((np.array(['Metrics']), np.array(metrics_df['Metric']))))
                initial = False
           
            table.append(np.concatenate((np.array([str(alpha)]), np.array(metrics_df['Mean']))))

    # Transposing the array
    data_transposed = np.array(table).T.tolist()

    # Writing to CSV
    filename = '.\\UpdatedPlots\\' + model + '_portfoliometrics_' + signal +'_v2.csv'
    
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data_transposed)
                

In [ ]:
# No Certainty
signals = ['1', '2', '3', '5', '6', 'OT']
model = 'arima_garch'
s = None


for signal in signals:

    initial = True
    table = []
    
    for strategy in ['2']:

        
            
        # File paths
        metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{str(alpha)}_s{s}_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

        # Read CSV files as DataFrames
        metrics_df = pd.read_csv(metrics_path)

        if initial:
            table.append(np.concatenate((np.array(['Metrics']), np.array(metrics_df['Metric']))))
            initial = False

        table.append(np.concatenate((np.array([str(alpha)]), np.array(metrics_df['Mean']))))

    # Transposing the array
    data_transposed = np.array(table).T.tolist()

    # Writing to CSV
    filename = '.\\UpdatedPlots\\' + model + '_portfoliometrics_' + signal +'_no_certainty.csv'
    
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data_transposed)
                

In [ ]:
# No Certainty 
signals = ['1', '2', '3', '5', '6', 'OT']
model = 'transformer_enbpi'
s = 5


for signal in signals:

    initial = True
    table = []
    
    for strategy in ['2']:

        
        # File paths
        metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\{model}_alpha{str(alpha)}_s{s}_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

        # Read CSV files as DataFrames
        metrics_df = pd.read_csv(metrics_path)

        if initial:
            table.append(np.concatenate((np.array(['Metrics']), np.array(metrics_df['Metric']))))
            initial = False

        table.append(np.concatenate((np.array([str(alpha)]), np.array(metrics_df['Mean']))))

    # Transposing the array
    data_transposed = np.array(table).T.tolist()

    # Writing to CSV
    filename = '.\\UpdatedPlots\\' + model + '_portfoliometrics_' + signal +'_no_certainty.csv'
    
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data_transposed)
                

In [ ]:
#Final tables:

signals = ['1', '2', '3', '5', '6', 'OT']
metrics = ['Cumulative Returns', 'Annualized Volatility', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown']
alpha = 0.01
initial = True
table = []

for signal in signals:

    
    
    for strategy in ['3']:

        for alpha_no in alphas:
            
            # File paths
            transformer_enbpi_metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\transformer_enbpi_alpha{alpha}_s20_optimisesharpe\\test_portfolio_metrics{strategy}.csv"
            arima_garch_metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\arima_garch_alpha{alpha}_sNone_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

            
            # Read CSV files as DataFrames
            transformer_enbpi_metrics_df = pd.read_csv(transformer_enbpi_metrics_path)
            arima_garch_metrics_df = pd.read_csv(arima_garch_metrics_path)
            
            transformer_enbpi_metrics_df.set_index('Metric', inplace=True)
            arima_garch_metrics_df.set_index('Metric', inplace=True)
            
    if initial:
        # Create alternating headers for ARIMA and Transformer metrics
        header = ['Signal']  # Start with 'Signal'
        header += [item for metric in metrics for item in (f'arima_{metric}', f'transformer_{metric}')]  # Correct list comprehension

        # Append the header to the table
        table.append(np.array(header))

        # Set initial to False after appending the header
        initial = False
        
    
    # Extract the metrics from both dataframes
    arima_garch_metrics = arima_garch_metrics_df['Mean'][metrics].values
    transformer_enbpi_metrics = transformer_enbpi_metrics_df['Mean'][metrics].values

    # Alternate between ARIMA and Transformer metrics
    alternating_metrics = np.array([val for pair in zip(arima_garch_metrics, transformer_enbpi_metrics) for val in pair])

    # Concatenate the signal with the alternating metrics
    row = np.concatenate(([signal], alternating_metrics))
    
    # Append the row to the table
    table.append(row)

filename = '.\\UpdatedPlots\\' + 'arima_garch_enbpi_portfoliometrics'+str(alpha)+'.csv'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(table)
    

In [ ]:
#Final tables (difference):

signals = ['1', '2', '3', '5', '6', 'OT']
metrics = ['Cumulative Returns', 'Annualized Volatility', 'Sharpe Ratio', 'Sortino Ratio', 'Maximum Drawdown']

initial = True
table = []

for signal in signals:

    
    
    for strategy in ['3']:

        for alpha in alphas:
            
            # File paths
            transformer_enbpi_metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\transformer_enbpi_alpha0.05_s20_optimisesharpe\\test_portfolio_metrics{strategy}.csv"
            arima_garch_metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\arima_garch_alpha0.05_sNone_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

            
            # Read CSV files as DataFrames
            transformer_enbpi_metrics_df = pd.read_csv(transformer_enbpi_metrics_path)
            arima_garch_metrics_df = pd.read_csv(arima_garch_metrics_path)
            
            transformer_enbpi_metrics_df.set_index('Metric', inplace=True)
            arima_garch_metrics_df.set_index('Metric', inplace=True)
            
    for strategy in ['2']:

        for alpha in alphas:
            
            # File paths
            no_cert_transformer_enbpi_metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\transformer_enbpi_alpha0.05_s20_optimisesharpe\\test_portfolio_metrics{strategy}.csv"
            no_cert_arima_garch_metrics_path = f"C:\\Users\\porte\\Downloads\\UpdatedPlots\\trading_model_data_{signal}\\arima_garch_alpha0.05_sNone_optimisesharpe\\test_portfolio_metrics{strategy}.csv"

            
            # Read CSV files as DataFrames
            no_cert_transformer_enbpi_metrics_df = pd.read_csv(no_cert_transformer_enbpi_metrics_path)
            no_cert_arima_garch_metrics_df = pd.read_csv(no_cert_arima_garch_metrics_path)
            
            no_cert_transformer_enbpi_metrics_df.set_index('Metric', inplace=True)
            no_cert_arima_garch_metrics_df.set_index('Metric', inplace=True)
            
    if initial:
        # Create alternating headers for ARIMA and Transformer metrics
        header = ['Signal']  # Start with 'Signal'
        header += [item for metric in metrics for item in (f'arima_{metric}', f'transformer_{metric}')]  # Correct list comprehension

        # Append the header to the table
        table.append(np.array(header))

        # Set initial to False after appending the header
        initial = False
        
    
    # Extract the metrics from both dataframes
    arima_garch_metrics = np.subtract(arima_garch_metrics_df['Mean'][metrics].values,no_cert_arima_garch_metrics_df['Mean'][metrics].values)
    transformer_enbpi_metrics = np.subtract(transformer_enbpi_metrics_df['Mean'][metrics].values,no_cert_transformer_enbpi_metrics_df['Mean'][metrics].values)

    # Alternate between ARIMA and Transformer metrics
    alternating_metrics = np.array([val for pair in zip(arima_garch_metrics, transformer_enbpi_metrics) for val in pair])

    # Concatenate the signal with the alternating metrics
    row = np.concatenate(([signal], alternating_metrics))
    
    # Append the row to the table
    table.append(row)

filename = '.\\UpdatedPlots\\' + 'difference_arima_garch_enbpi_portfoliometrics.csv'

with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(table)
    